In [1]:
import numpy as np

In [2]:
def get_subWordSet(input_string):
    length = len(input_string)
    return set([input_string[i: j + 1] for i in range(length) for j in range(i, length)])


def get_weightedJac(R1, R2, 
                    get_weight=None, *, 
                    symbol_weight_map=None, 
                    verbose=False):
    """
    Calculate the weighted subword Jaccard distance between two formatted medical histories.
    """
    if get_weight is None:
        get_weight = lambda seq: 1
    
    if get_weight == 'linear':
        get_weight = lambda seq: np.array([symbol_weight_map[s] for s in seq]).sum()
    
    S1 = get_subWordSet(R1)
    S2 = get_subWordSet(R2)
    union = S1.union(S2)
    intersection = S1.intersection(S2)
    
    if verbose:
        print('S1: {}'.format(S1))
        print('S2: {}'.format(S2))
        print('Union: {}'.format(union))
        print('Intersection: {}'.format(intersection))
    
    weight_union = 0
    weight_intersection = 0
    for seq in union:
        weight = get_weight(seq)
        weight_union += weight
        if seq in intersection:
            weight_intersection += weight
    
    return 1.0 - float(weight_intersection) / float(weight_union)

In [3]:
def get_all_weightedJac(R1, R2, 
                        get_weight=None, *,
                        end_time=None,
                        symbol_weight_map=None, 
                        verbose=False):
    """
    Let R1 and R2 be two medical histories.
    The function calculate weighted subword Jaccard distance
    between R1[:end_time] and R2[:end_time],
    for all end_time in range(1, min(len(R1), len(R2))).
    
    The reason for designing this function is that
    Jaccard distance between R1[:end_time], R2[:end_time])
    can be calcucated dynamically using memoization.
    (search dynamic programming for more detail on
    memoization).
    """
    if get_weight is None:
        get_weight = lambda seq: 1
    
    if get_weight == 'linear':
        get_weight = lambda seq: np.array([symbol_weight_map[s] for s in seq]).sum()
    
    if end_time == None:
        end_time = min(len(R1), len(R2))
        
    union = {}
    weight_union = 0
    weight_intersection = 0
    distances = []
    
    for time in range(end_time):
        
        for t in range(time + 1):
            word1 = R1[time - t: time + 1]
            word2 = R2[time - t: time + 1]
            
            if word1 in union:
                if union[word1][0] == '2':
                    union[word1][0] = 'b' # 'b' is a shorthand for both
                    weight_intersection += union[word1][1]
            else:
                weight = get_weight(word1)
                union[word1] = ['1', weight]
                weight_union += weight
                
            if word2 in union:
                if union[word2][0] == '1':
                    union[word2][0] = 'b'
                    weight_intersection += union[word2][1]
            else:
                weight = get_weight(word2)
                union[word2] = ['2', weight]
                weight_union += weight
        
        
        distance = 1.0 - float(weight_intersection) / float(weight_union)
        distances.append(distance)
        
        if verbose:
            print('record length = {}'.format(time))
            # print('\tunion = {}'.format(union))
            print('\tweight of union = {}'.format(weight_union))
            print('\tweight of intersection = {}'.format(weight_intersection))
            print('\tweighted subword Jaccard distance = {}\n'.format(distance))
            
    return np.array(distances)

### Test on get_all_weightedJac

In [4]:
symbol_weight_map = {'1': 1, '2': 2}
R1 = '1222111212221'
R2 = '12121222111121'
# get_all_weightedJac(R1, R2, get_weight='linear', symbol_weight_map=symbol_weight_map, verbose=True)

In [5]:
def get_all_weightedJac_matrices(histories, *,
                                 get_weight=None,
                                 symbol_weight_map=None,
                                 end_time=None,
                                 verbose=False):
    
    """
    """

    length = np.array([len(h) for h in histories]).min()
    
    if end_time is None:
        end_time = length
        
    if end_time > length:
        raise Exception('end_time must be smaller or equal to the length of shortest history!')
    
    num_histories = len(histories)
    matrices = np.zeros((end_time, num_histories, num_histories))
    for i in range(num_histories):
        for j in range(i + 1, num_histories):
            dist = get_all_weightedJac(histories[i], histories[j], 
                                       get_weight=get_weight, 
                                       end_time=end_time, 
                                       symbol_weight_map=symbol_weight_map)
            for t in range(len(dist)):
                matrices[t][i][j] = dist[t]
                matrices[t][j][i] = dist[t]
    
    return matrices

# get_neighborhood_prevalence function

In [6]:
def get_neighborhood_prevalence(dist, epsilon, histories, time):
    
    """
    For all i in range(len(histories)), 
    calculate the prevalence of histories[i][time] 
    in the epsilon-neighborhood of histories[i][: time-1].
    """
    
    dim = len(dist)
    neighborhood_prevalence = np.zeros(dim)
    for i in range(dim):
        count = 0
        target = histories[i][time]
        indices = np.arange(dim)[dist[i] <= epsilon]
        
        for idx in indices:
            if histories[idx][time] == target:
                neighborhood_prevalence[i] += 1.
        
        neighborhood_prevalence[i] /= float(len(indices))    
    
    return neighborhood_prevalence

# Iterative approach

In [26]:
def get_modified_Jac(Jac, prob):
    
    """
    """
    
    modified_Jac = np.zeros(Jac.shape)
    for i in range(len(prob)):
        for j in range(i + 1, len(prob)):
            temp = (1. / prob[i] + 1. / prob[j] - 2.) * Jac[i][j]
            modified_Jac[i][j] = temp
            modified_Jac[j][i] = temp
    
    return modified_Jac
    
    
def get_distance(histories, *, 
                 epsilon, 
                 prevalence,
                 get_weight=None, 
                 symbol_weight_map=None, 
                 end_time=None):
    
    """
    """
    
    length = np.array([len(h) for h in histories]).min()
    
    if end_time is None:
        end_time = length
        
    if end_time > length:
        raise Exception('end_time must be smaller or equal to the length of shortest history!')
    
    Jacs = get_all_weightedJac_matrices(histories, 
                                        get_weight=get_weight,
                                        symbol_weight_map=symbol_weight_map,
                                        end_time=end_time,
                                        verbose=False)
    
    prob = np.array([prevalence[history[0]] for history in histories])

    modified_Jacs = []
    
    for t in range(end_time - 1):
        modified_Jac = get_modified_Jac(Jacs[t], prob)
        modified_Jacs.append(modified_Jac)
        prob = get_neighborhood_prevalence(modified_Jac, epsilon, histories, t + 1)
    
    modified_Jacs.append(get_modified_Jac(Jacs[end_time - 1], prob))
    
    return modified_Jacs

# Generate dummy history

In [27]:
length = 50 # Let us assume that histories are of uniform length
num_histories = 20
num_diseases = 10
prevalence = np.array([50, 1, .5, .5, .5, .2, .2, .1, .1, 0.05])
prevalence = prevalence / prevalence.sum()

## The no-brainer way
with which I probably will give a man breast cancer or so... (or a woman prostate cancer, which is even less likely)

**Please help me come up more sensable ways of generating dummy medical histories.**

In [28]:
matrix = np.random.choice(10, size=(num_histories, length), replace=True, p=prevalence)
histories = []
for idx, row in enumerate(matrix):
    history = ''.join(list(map(str, row)))
    print('{}:\t{}'.format(idx, history))
    histories.append(history)

0:	00000005000010000000400000000000000000000000000200
1:	00000000000031000000000000000000000000000000000000
2:	00000000301000001000000000000000000201100010000000
3:	30000000000000000400000000000000000000000000000000
4:	00003000300000000000004000000000050000000000000000
5:	00000500000000000000000000000000000004000000001000
6:	00000000000000000000000000003000000000000020001000
7:	00000004000000000000000000000000000000000000000000
8:	00000000010000000000005000000000000000000000000000
9:	00000000000000000000000000000000003000000000000000
10:	00000000000000000000000000000000000000060000000000
11:	00000000009100003000000050030000000000000000000000
12:	00000000000000000000000004000800001020000000000100
13:	00000005000000000060000000000000000000000000000000
14:	00000000000000000000000000000000000000000000000000
15:	00000040000004000000000000010000000040110000000000
16:	00003000000000000000000000000000070000000000000000
17:	00000000000000000000000000000000000000000000000000
18:	0000100000000000

In [29]:
symbol_weight_map = {str(i) : 1. / prevalence[i] for i in range(num_diseases)}
Jacs = get_all_weightedJac_matrices(histories,
                                    get_weight='linear',
                                    symbol_weight_map=symbol_weight_map,
                                    end_time=None)

prevalence_dict = {str(i): prevalence[i] for i in range(num_diseases)}
modified_Jacs = get_distance(histories, 
                             epsilon=1000, 
                             prevalence=prevalence_dict, 
                             get_weight='linear', 
                             symbol_weight_map=symbol_weight_map)

In [31]:
# Jacs
modified_Jacs

[array([[  0.   ,   0.   ,   0.   , 105.363,   0.   ,   0.   ,   0.   ,
           0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
           0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ],
        [  0.   ,   0.   ,   0.   , 105.363,   0.   ,   0.   ,   0.   ,
           0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
           0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ],
        [  0.   ,   0.   ,   0.   , 105.363,   0.   ,   0.   ,   0.   ,
           0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
           0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ],
        [105.363, 105.363, 105.363,   0.   , 105.363, 105.363, 105.363,
         105.363, 105.363, 105.363, 105.363, 105.363, 105.363, 105.363,
         105.363, 105.363, 105.363, 105.363, 105.363, 105.363],
        [  0.   ,   0.   ,   0.   , 105.363,   0.   ,   0.   ,   0.   ,
           0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
           0.   ,   0.  

# Cluster and plot

In [38]:
from sklearn.cluster import KMeans, spectral_clustering
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
beta = 1.
sim = np.exp(-beta * Jaccard / Jaccard.std())
labels = spectral_clustering(sim, n_clusters=10, eigen_solver='arpack')
sortedIdx = np.argsort(labels)
Jsorted=Jaccard[sortedIdx][:,sortedIdx]

sns.heatmap(Jsorted)
plt.show()